In [1]:
experiment_name = 'sm'

In [2]:
from pathlib import Path
from fastai.data_block import get_files
import pickle
import torch
import numpy as np, pandas as pd

In [3]:
class defaultlist(list):
    def __init__(self, fx):
        self._fx = fx
    def _fill(self, index):
        while len(self) <= index:
            self.append(self._fx())
    def __setitem__(self, index, value):
        self._fill(index)
        list.__setitem__(self, index, value)
    def __getitem__(self, index):
        self._fill(index)
        return list.__getitem__(self, index)

In [4]:
data_dir = Path('data/predictions')
base_classes = [ 'any', 'epidural', 'intraparenchymal', 'intraventricular', 'subarachnoid', 'subdural' ]

In [5]:
stage = "stage_2"

In [6]:
fn_to_study_ix = pickle.load(open(f'data/{stage}_fn_to_study_ix.pickle', 'rb'))
study_ix_to_fn = pickle.load(open(f'data/{stage}_study_ix_to_fn.pickle', 'rb'))
study_to_data = pickle.load(open(f'data/{stage}_study_to_data.pickle', 'rb'))
df = pd.read_csv(f"data/{stage}_train_dicom_diags_norm.csv")
df = df.set_index(['fid'])


In [7]:
labels_df = df.loc[:,'any':'subdural']
labels_df.index.name = 'fn'

In [8]:
csvs = sorted([str(fn) for fn in get_files(data_dir, extensions='.csv')])
csv = csvs[0]
def where_test(csv):
    res = csv.find('_test') 
    return res if res!= -1 else csv.find('_valid') 
where_fold = lambda csv: csv.index('fold')
where_stem = lambda csv: len(csv)-csv[::-1].index('/')
where_sz   = lambda csv: csv.index('_sz')
model_fns = sorted(np.unique([csv[:where_test(csv)] for csv in csvs]),
                   key=lambda x:x[where_stem(x):where_sz(x)]+x[where_fold(x):where_test(x)])
archs     = np.unique([model_fn[:where_sz(model_fn)] for model_fn in model_fns]).tolist()
n_folds = len(model_fns)//len(archs)
model_fns, archs, 

(['data/predictions/densenet121-53_train_cpu_window_uint8_sz512_cv0.0413_subdural_loss_fold1_of_5',
  'data/predictions/densenet121-53_train_cpu_window_uint8_sz512_cv0.0406_subdural_loss_fold2_of_5',
  'data/predictions/densenet121-53_train_cpu_window_uint8_sz512_cv0.0403_subdural_loss_fold3_of_5',
  'data/predictions/densenet121-53_train_cpu_window_uint8_sz512_cv0.0399_subdural_loss_fold4_of_5',
  'data/predictions/densenet121-53_train_cpu_window_uint8_sz512_cv0.0386_subdural_loss_fold5_of_5',
  'data/predictions/densenet121_sz512_cv0.0749_weighted_loss_fold1_of_5',
  'data/predictions/densenet121_sz512_cv0.0743_weighted_loss_fold2_of_5',
  'data/predictions/densenet121_sz512_cv0.0748_weighted_loss_fold3_of_5',
  'data/predictions/densenet121_sz512_cv0.0738_weighted_loss_fold4_of_5',
  'data/predictions/densenet121_sz512_cv0.0739_weighted_loss_fold5_of_5',
  'data/predictions/resnet101-53_train_cpu_window_uint8_sz512_cv0.0375_subdural_loss_fold1_of_5',
  'data/predictions/resnet101-53

In [9]:
arch_names = [arch[where_stem(arch):] for arch in archs]
test_folds_dfs, train_arch_dfs = defaultlist(list),[]
for arch in archs:
    train_model_dfs = []
    _arch = arch[where_stem(arch):]
    for i,model_fn in enumerate([model for model in model_fns if model.find(arch+"_")!=-1]):
        print(i,model_fn)
        t_df = pd.read_csv(model_fn + '_valid_fns.csv')
        t = torch.load(model_fn + '_valid.pth')
        t_mean, t_std = t.mean(dim=0), t.std(dim=0)
        train_model_dfs.append(pd.DataFrame({**{'fn' : t_df.values.squeeze(-1),},
                      **{f'X{_arch}_{c}_mean' : t_mean[:,6+i].numpy() for i,c in enumerate(base_classes)},
                      **{f'X{_arch}_{c}_std'  :  t_std[:,6+i].numpy() for i,c in enumerate(base_classes)}}))
        
        t_df = pd.read_csv(model_fn + '_test_fns.csv')
        t = torch.load(model_fn + '_test.pth')
        t_mean, t_std = t.mean(dim=0), t.std(dim=0)
        test_folds_dfs[i].append(pd.DataFrame({**{'fn' : t_df.values.squeeze(-1),},
                      **{f'X{_arch}_{c}_mean' : t_mean[:,6+i].numpy() for i,c in enumerate(base_classes)},
                      **{f'X{_arch}_{c}_std'  :  t_std[:,6+i].numpy() for i,c in enumerate(base_classes)}}))

    train_arch_dfs.append(pd.concat(train_model_dfs, axis=0))

0 data/predictions/densenet121_sz512_cv0.0749_weighted_loss_fold1_of_5
1 data/predictions/densenet121_sz512_cv0.0743_weighted_loss_fold2_of_5
2 data/predictions/densenet121_sz512_cv0.0748_weighted_loss_fold3_of_5
3 data/predictions/densenet121_sz512_cv0.0738_weighted_loss_fold4_of_5
4 data/predictions/densenet121_sz512_cv0.0739_weighted_loss_fold5_of_5
0 data/predictions/densenet121-53_train_cpu_window_uint8_sz512_cv0.0413_subdural_loss_fold1_of_5
1 data/predictions/densenet121-53_train_cpu_window_uint8_sz512_cv0.0406_subdural_loss_fold2_of_5
2 data/predictions/densenet121-53_train_cpu_window_uint8_sz512_cv0.0403_subdural_loss_fold3_of_5
3 data/predictions/densenet121-53_train_cpu_window_uint8_sz512_cv0.0399_subdural_loss_fold4_of_5
4 data/predictions/densenet121-53_train_cpu_window_uint8_sz512_cv0.0386_subdural_loss_fold5_of_5
0 data/predictions/resnet101_sz512_cv0.0758_weighted_loss_fold1_of_5
1 data/predictions/resnet101_sz512_cv0.0754_weighted_loss_fold2_of_5
2 data/predictions/res

In [10]:
tta_df = train_arch_dfs[0]
for arch_df in train_arch_dfs[1:]:
    tta_df = tta_df.merge(arch_df,on='fn')

In [11]:
tta_df = tta_df.merge(labels_df,on='fn')

In [12]:
tta_df.head()

,fn,Xdensenet121_any_mean,Xdensenet121_epidural_mean,Xdensenet121_intraparenchymal_mean,Xdensenet121_intraventricular_mean,Xdensenet121_subarachnoid_mean,Xdensenet121_subdural_mean,Xdensenet121_any_std,Xdensenet121_epidural_std,Xdensenet121_intraparenchymal_std,...,Xresnext50_32x4d-53_train_cpu_window_uint8_intraparenchymal_std,Xresnext50_32x4d-53_train_cpu_window_uint8_intraventricular_std,Xresnext50_32x4d-53_train_cpu_window_uint8_subarachnoid_std,Xresnext50_32x4d-53_train_cpu_window_uint8_subdural_std,any,epidural,intraparenchymal,intraventricular,subarachnoid,subdural
0,ID_0000950d7,0.000216,0.000009,0.000040,0.000017,0.000131,0.000070,0.000077,2.602864e-06,0.000013,...,0.000009,2.093691e-06,0.000017,0.000002,0,0,0,0,0,0
1,ID_0000aee4b,0.000652,0.000132,0.000049,0.000030,0.000237,0.000415,0.000478,7.363618e-05,0.000018,...,0.000003,4.035392e-07,0.000253,0.000225,0,0,0,0,0,0
2,ID_0001de0e8,0.000875,0.000079,0.000283,0.000013,0.000218,0.000636,0.001155,8.438517e-05,0.000151,...,0.000247,1.205952e-05,0.001305,0.000090,0,0,0,0,0,0
3,ID_0002003a8,0.001583,0.000008,0.000745,0.000176,0.000233,0.000478,0.000657,8.040458e-07,0.000355,...,0.000542,3.861410e-04,0.000488,0.000947,0,0,0,0,0,0
4,ID_000229f2a,0.000251,0.000041,0.000084,0.000039,0.000162,0.000084,0.000072,1.584314e-05,0.000014,...,0.001312,1.682060e-06,0.000491,0.000120,0,0,0,0,0,0


In [13]:
len(tta_df), len(labels_df)

(752802, 752802)

# Begin boosting data preparation

In [14]:
#add study and z position to dataframe to join
tta_df['study'] = tta_df.fn.apply(lambda fn: fn_to_study_ix[fn][0] )
tta_df['z'] = tta_df.fn.apply(lambda fn: int(fn_to_study_ix[fn][1]) )
tta_df.head()

,fn,Xdensenet121_any_mean,Xdensenet121_epidural_mean,Xdensenet121_intraparenchymal_mean,Xdensenet121_intraventricular_mean,Xdensenet121_subarachnoid_mean,Xdensenet121_subdural_mean,Xdensenet121_any_std,Xdensenet121_epidural_std,Xdensenet121_intraparenchymal_std,...,Xresnext50_32x4d-53_train_cpu_window_uint8_subarachnoid_std,Xresnext50_32x4d-53_train_cpu_window_uint8_subdural_std,any,epidural,intraparenchymal,intraventricular,subarachnoid,subdural,study,z
0,ID_0000950d7,0.000216,0.000009,0.000040,0.000017,0.000131,0.000070,0.000077,2.602864e-06,0.000013,...,0.000017,0.000002,0,0,0,0,0,0,ID_84296c3845,32
1,ID_0000aee4b,0.000652,0.000132,0.000049,0.000030,0.000237,0.000415,0.000478,7.363618e-05,0.000018,...,0.000253,0.000225,0,0,0,0,0,0,ID_1e59488a44,7
2,ID_0001de0e8,0.000875,0.000079,0.000283,0.000013,0.000218,0.000636,0.001155,8.438517e-05,0.000151,...,0.001305,0.000090,0,0,0,0,0,0,ID_9d97cf289b,22
3,ID_0002003a8,0.001583,0.000008,0.000745,0.000176,0.000233,0.000478,0.000657,8.040458e-07,0.000355,...,0.000488,0.000947,0,0,0,0,0,0,ID_805824f65e,17
4,ID_000229f2a,0.000251,0.000041,0.000084,0.000039,0.000162,0.000084,0.000072,1.584314e-05,0.000014,...,0.000491,0.000120,0,0,0,0,0,0,ID_f54eba3225,30


In [15]:
#now join top and bottom
tta_df['z+'] = tta_df.z+1
tta_df['z-'] = tta_df.z-1
#only merge means to avoid too many columns
merge_cols = [c for c in tta_df.columns if 'mean' in c] 
merged_df = pd.merge(tta_df, tta_df[merge_cols+ ['study','z+']], how='left', left_on=['study', 'z'], right_on=['study','z+'], suffixes=('','+1'))
merged_df = pd.merge(merged_df, tta_df[merge_cols+ ['study','z-']], how='left', left_on=['study', 'z'], right_on=['study','z-'], suffixes=('','-1'))
merged_df.drop(['z++1','z--1'], axis=1, inplace=True) #after merge these columns are not needed
len(merged_df)
merged_df.head()


,fn,Xdensenet121_any_mean,Xdensenet121_epidural_mean,Xdensenet121_intraparenchymal_mean,Xdensenet121_intraventricular_mean,Xdensenet121_subarachnoid_mean,Xdensenet121_subdural_mean,Xdensenet121_any_std,Xdensenet121_epidural_std,Xdensenet121_intraparenchymal_std,...,Xresnet50_intraparenchymal_mean-1,Xresnet50_intraventricular_mean-1,Xresnet50_subarachnoid_mean-1,Xresnet50_subdural_mean-1,Xresnext50_32x4d-53_train_cpu_window_uint8_any_mean-1,Xresnext50_32x4d-53_train_cpu_window_uint8_epidural_mean-1,Xresnext50_32x4d-53_train_cpu_window_uint8_intraparenchymal_mean-1,Xresnext50_32x4d-53_train_cpu_window_uint8_intraventricular_mean-1,Xresnext50_32x4d-53_train_cpu_window_uint8_subarachnoid_mean-1,Xresnext50_32x4d-53_train_cpu_window_uint8_subdural_mean-1
0,ID_0000950d7,0.000216,0.000009,0.000040,0.000017,0.000131,0.000070,0.000077,2.602864e-06,0.000013,...,0.000003,2.410099e-07,0.000009,0.000010,0.000023,0.000012,0.000041,0.000015,0.000040,0.000004
1,ID_0000aee4b,0.000652,0.000132,0.000049,0.000030,0.000237,0.000415,0.000478,7.363618e-05,0.000018,...,0.000032,2.871577e-05,0.000192,0.000307,0.000320,0.000031,0.000005,0.000004,0.000129,0.000094
2,ID_0001de0e8,0.000875,0.000079,0.000283,0.000013,0.000218,0.000636,0.001155,8.438517e-05,0.000151,...,0.000386,4.940505e-05,0.001212,0.001620,0.002983,0.000075,0.000413,0.000028,0.001168,0.000841
3,ID_0002003a8,0.001583,0.000008,0.000745,0.000176,0.000233,0.000478,0.000657,8.040458e-07,0.000355,...,0.000253,2.354449e-05,0.000079,0.000145,0.000816,0.000072,0.000268,0.000263,0.000419,0.000390
4,ID_000229f2a,0.000251,0.000041,0.000084,0.000039,0.000162,0.000084,0.000072,1.584314e-05,0.000014,...,0.000087,1.064072e-05,0.000406,0.000361,0.000489,0.000057,0.000044,0.000004,0.000202,0.000139


In [16]:
#now add maximum value for each merge_col by study
group_df = merged_df[merge_cols+['study']].groupby('study').agg('max')
merged_df = pd.merge(merged_df, group_df, how='left', on='study', suffixes=('','_max'))
merged_df.head()

,fn,Xdensenet121_any_mean,Xdensenet121_epidural_mean,Xdensenet121_intraparenchymal_mean,Xdensenet121_intraventricular_mean,Xdensenet121_subarachnoid_mean,Xdensenet121_subdural_mean,Xdensenet121_any_std,Xdensenet121_epidural_std,Xdensenet121_intraparenchymal_std,...,Xresnet50_intraparenchymal_mean_max,Xresnet50_intraventricular_mean_max,Xresnet50_subarachnoid_mean_max,Xresnet50_subdural_mean_max,Xresnext50_32x4d-53_train_cpu_window_uint8_any_mean_max,Xresnext50_32x4d-53_train_cpu_window_uint8_epidural_mean_max,Xresnext50_32x4d-53_train_cpu_window_uint8_intraparenchymal_mean_max,Xresnext50_32x4d-53_train_cpu_window_uint8_intraventricular_mean_max,Xresnext50_32x4d-53_train_cpu_window_uint8_subarachnoid_mean_max,Xresnext50_32x4d-53_train_cpu_window_uint8_subdural_mean_max
0,ID_0000950d7,0.000216,0.000009,0.000040,0.000017,0.000131,0.000070,0.000077,2.602864e-06,0.000013,...,0.486492,0.905775,0.150154,0.099021,0.857974,0.001780,0.361130,0.789858,0.168123,0.073204
1,ID_0000aee4b,0.000652,0.000132,0.000049,0.000030,0.000237,0.000415,0.000478,7.363618e-05,0.000018,...,0.002535,0.002878,0.006184,0.003424,0.027332,0.000609,0.003878,0.008006,0.009463,0.006090
2,ID_0001de0e8,0.000875,0.000079,0.000283,0.000013,0.000218,0.000636,0.001155,8.438517e-05,0.000151,...,0.002166,0.003913,0.004683,0.005613,0.018091,0.002842,0.012398,0.009452,0.009165,0.005672
3,ID_0002003a8,0.001583,0.000008,0.000745,0.000176,0.000233,0.000478,0.000657,8.040458e-07,0.000355,...,0.001896,0.001544,0.004229,0.002354,0.021156,0.000476,0.005249,0.028012,0.004112,0.020228
4,ID_000229f2a,0.000251,0.000041,0.000084,0.000039,0.000162,0.000084,0.000072,1.584314e-05,0.000014,...,0.011392,0.059925,0.010892,0.015900,0.042269,0.001678,0.012257,0.029033,0.014108,0.011063


In [17]:
#manage NANs created by merging
merged_cols = [c for c in merged_df.columns if c.endswith('1')]

for c in merged_cols:
    merged_df[c].fillna(merged_df[c.replace('-1', '').replace('+1', '')], inplace=True)
    
merged_df.head()

,fn,Xdensenet121_any_mean,Xdensenet121_epidural_mean,Xdensenet121_intraparenchymal_mean,Xdensenet121_intraventricular_mean,Xdensenet121_subarachnoid_mean,Xdensenet121_subdural_mean,Xdensenet121_any_std,Xdensenet121_epidural_std,Xdensenet121_intraparenchymal_std,...,Xresnet50_intraparenchymal_mean_max,Xresnet50_intraventricular_mean_max,Xresnet50_subarachnoid_mean_max,Xresnet50_subdural_mean_max,Xresnext50_32x4d-53_train_cpu_window_uint8_any_mean_max,Xresnext50_32x4d-53_train_cpu_window_uint8_epidural_mean_max,Xresnext50_32x4d-53_train_cpu_window_uint8_intraparenchymal_mean_max,Xresnext50_32x4d-53_train_cpu_window_uint8_intraventricular_mean_max,Xresnext50_32x4d-53_train_cpu_window_uint8_subarachnoid_mean_max,Xresnext50_32x4d-53_train_cpu_window_uint8_subdural_mean_max
0,ID_0000950d7,0.000216,0.000009,0.000040,0.000017,0.000131,0.000070,0.000077,2.602864e-06,0.000013,...,0.486492,0.905775,0.150154,0.099021,0.857974,0.001780,0.361130,0.789858,0.168123,0.073204
1,ID_0000aee4b,0.000652,0.000132,0.000049,0.000030,0.000237,0.000415,0.000478,7.363618e-05,0.000018,...,0.002535,0.002878,0.006184,0.003424,0.027332,0.000609,0.003878,0.008006,0.009463,0.006090
2,ID_0001de0e8,0.000875,0.000079,0.000283,0.000013,0.000218,0.000636,0.001155,8.438517e-05,0.000151,...,0.002166,0.003913,0.004683,0.005613,0.018091,0.002842,0.012398,0.009452,0.009165,0.005672
3,ID_0002003a8,0.001583,0.000008,0.000745,0.000176,0.000233,0.000478,0.000657,8.040458e-07,0.000355,...,0.001896,0.001544,0.004229,0.002354,0.021156,0.000476,0.005249,0.028012,0.004112,0.020228
4,ID_000229f2a,0.000251,0.000041,0.000084,0.000039,0.000162,0.000084,0.000072,1.584314e-05,0.000014,...,0.011392,0.059925,0.010892,0.015900,0.042269,0.001678,0.012257,0.029033,0.014108,0.011063


In [18]:
#add fold for training
merged_df['fold'] = merged_df.study.apply(lambda s: study_to_data[s]['fold'])
merged_df.fold.value_counts()

2.0     40785
0.0     40527
3.0     40439
1.0     40068
5.0     39983
4.0     39953
11.0    39882
6.0     39779
9.0     39644
7.0     39625
10.0    39378
8.0     39360
16.0    39355
13.0    39301
12.0    39183
17.0    39136
15.0    38904
14.0    38781
18.0    38719
Name: fold, dtype: int64

# Prepare test data frame

In [19]:
merge_cols

['Xdensenet121_any_mean',
 'Xdensenet121_epidural_mean',
 'Xdensenet121_intraparenchymal_mean',
 'Xdensenet121_intraventricular_mean',
 'Xdensenet121_subarachnoid_mean',
 'Xdensenet121_subdural_mean',
 'Xdensenet121-53_train_cpu_window_uint8_any_mean',
 'Xdensenet121-53_train_cpu_window_uint8_epidural_mean',
 'Xdensenet121-53_train_cpu_window_uint8_intraparenchymal_mean',
 'Xdensenet121-53_train_cpu_window_uint8_intraventricular_mean',
 'Xdensenet121-53_train_cpu_window_uint8_subarachnoid_mean',
 'Xdensenet121-53_train_cpu_window_uint8_subdural_mean',
 'Xresnet101_any_mean',
 'Xresnet101_epidural_mean',
 'Xresnet101_intraparenchymal_mean',
 'Xresnet101_intraventricular_mean',
 'Xresnet101_subarachnoid_mean',
 'Xresnet101_subdural_mean',
 'Xresnet101-53_train_cpu_window_uint8_any_mean',
 'Xresnet101-53_train_cpu_window_uint8_epidural_mean',
 'Xresnet101-53_train_cpu_window_uint8_intraparenchymal_mean',
 'Xresnet101-53_train_cpu_window_uint8_intraventricular_mean',
 'Xresnet101-53_train_

In [20]:
#fill and create test dataframe
test_dfs = []

for i in range(len(test_folds_dfs)):
    current_test_fold = test_folds_dfs[i]
    test_df = current_test_fold[0]
    for arch_df in current_test_fold[1:]:
        test_df = test_df.merge(arch_df,on='fn')
    test_dfs.append(test_df)

for i in range(len(test_dfs)):
    #add study and z position to dataframe to join
    test_dfs[i]['study'] = test_dfs[i].fn.apply(lambda fn: fn_to_study_ix[fn][0] )
    test_dfs[i]['z'] = test_dfs[i].fn.apply(lambda fn: int(fn_to_study_ix[fn][1]) )
    
    #now join top and bottom
    test_dfs[i]['z+'] = test_dfs[i].z+1
    test_dfs[i]['z-'] = test_dfs[i].z-1
    
    #only merge means to avoid too many columns
    merge_cols = [c for c in tta_df.columns if 'mean' in c] 
    test_dfs[i] = pd.merge(test_dfs[i], test_dfs[i][merge_cols+ ['study','z+']], how='left', left_on=['study', 'z'], right_on=['study','z+'], suffixes=('','+1'))
    test_dfs[i] = pd.merge(test_dfs[i], test_dfs[i][merge_cols+ ['study','z-']], how='left', left_on=['study', 'z'], right_on=['study','z-'], suffixes=('','-1'))
    test_dfs[i].drop(['z++1','z--1'], axis=1, inplace=True) #after merge these columns are not needed
    
    #now add maximum value for each merge_col by study
    group_df = test_dfs[i][merge_cols+['study']].groupby('study').agg('max')
    test_dfs[i] = pd.merge(test_dfs[i], group_df, how='left', on='study', suffixes=('','_max'))
    
    #manage NANs created by merging
    merged_cols = [c for c in test_dfs[i].columns if c.endswith('1')]

    for c in merged_cols:
        test_dfs[i][c].fillna(test_dfs[i][c.replace('-1', '').replace('+1', '')], inplace=True)

test_dfs[0].head()

,fn,Xdensenet121_any_mean,Xdensenet121_epidural_mean,Xdensenet121_intraparenchymal_mean,Xdensenet121_intraventricular_mean,Xdensenet121_subarachnoid_mean,Xdensenet121_subdural_mean,Xdensenet121_any_std,Xdensenet121_epidural_std,Xdensenet121_intraparenchymal_std,...,Xresnet50_intraparenchymal_mean_max,Xresnet50_intraventricular_mean_max,Xresnet50_subarachnoid_mean_max,Xresnet50_subdural_mean_max,Xresnext50_32x4d-53_train_cpu_window_uint8_any_mean_max,Xresnext50_32x4d-53_train_cpu_window_uint8_epidural_mean_max,Xresnext50_32x4d-53_train_cpu_window_uint8_intraparenchymal_mean_max,Xresnext50_32x4d-53_train_cpu_window_uint8_intraventricular_mean_max,Xresnext50_32x4d-53_train_cpu_window_uint8_subarachnoid_mean_max,Xresnext50_32x4d-53_train_cpu_window_uint8_subdural_mean_max
0,ID_0fbf6a978,0.652952,0.064774,0.077681,0.005225,0.292178,0.338067,0.096585,3.203979e-02,0.054162,...,0.996345,0.973147,0.959216,0.376672,0.998088,0.032759,0.990556,0.993277,0.982379,0.296742
1,ID_d62ec3412,0.000902,0.000007,0.000197,0.000157,0.000337,0.000279,0.000446,3.052288e-06,0.000050,...,0.005606,0.011685,0.056988,0.012356,0.180219,0.004658,0.034183,0.070213,0.048766,0.028650
2,ID_cb544194b,0.013207,0.000106,0.000226,0.000060,0.003247,0.008092,0.005803,3.079419e-05,0.000056,...,0.003033,0.001944,0.181010,0.168275,0.316041,0.002888,0.028160,0.006408,0.157353,0.208944
3,ID_0d62513ec,0.004513,0.000018,0.000599,0.000379,0.001197,0.002964,0.001034,5.030724e-06,0.000253,...,0.001040,0.001171,0.003586,0.014528,0.013856,0.000814,0.003383,0.003476,0.002704,0.016037
4,ID_fc45b2151,0.000032,0.000001,0.000014,0.000009,0.000016,0.000031,0.000014,3.315683e-07,0.000004,...,0.001387,0.001326,0.004180,0.003761,0.019404,0.000497,0.003189,0.000700,0.010232,0.004891


# Train XG Boost

In [21]:
folds = [1,5,12] #three random folds

train_df = merged_df.query('fold not in @folds')
val_df = merged_df.query('fold in @folds')

len(train_df),len(val_df)


(633568, 119234)

In [22]:
#first check log loss for current model predictions
from sklearn.metrics import log_loss

resnet34_vars = [c for c in merged_df.columns if c.startswith('Xresnet34') and c.endswith('mean')]
y_true = val_df[base_classes]
y_pred = val_df[resnet34_vars]

for tc,pc in zip(y_true.columns,y_pred.columns):
    loss = log_loss( y_true[tc], y_pred[pc] )
    print(tc,pc,loss)


any Xresnet34_any_mean 0.10142146307769834
epidural Xresnet34_epidural_mean 0.012571424002310823
intraparenchymal Xresnet34_intraparenchymal_mean 0.04222755602983461
intraventricular Xresnet34_intraventricular_mean 0.025140569354671405
subarachnoid Xresnet34_subarachnoid_mean 0.06708897203821126
subdural Xresnet34_subdural_mean 0.08416194895275723


In [23]:
import xgboost as xgb
import os
from sklearn.multioutput import MultiOutputRegressor
#select only variables from deep learning model
model_vars = [c  for c in merged_df.columns if c.startswith('X')]
model_vars.append('z')

In [24]:
x_train = train_df[model_vars]
y_train = train_df[base_classes]

In [25]:
xgb_clf = MultiOutputRegressor(xgb.XGBRegressor(objective='binary:logistic',tree_method='gpu_hist',
                                            n_estimators=60, reg_lambda=14.0005, 
                                            max_depth=5,learning_rate=0.1188,gamma=0.0, reg_alpha=0.2043,
                                            min_child_weight=0.0,max_delta_step=0.0,subsample=1.0,colsample_bytree=1.0,
                                            colsample_bylevel=1.0,silent=0,nthread=-1,
                                            scale_pos_weight=1.0,base_score=0.05,seed=1337,missing=None,))
print('Training xgboost')
xgb_clf.fit(x_train, y_train) 

#save trained xgb model
pickle.dump(xgb_clf, open('data/xgb_%s.pickle'%experiment_name, 'wb'))

print('Done training')

Training xgboost
Done training


In [26]:
from catboost import CatBoostClassifier

In [27]:
cb_clf = MultiOutputRegressor(CatBoostClassifier(task_type='GPU',depth=5,l2_leaf_reg=30,iterations=500))
cb_clf.fit(x_train,y_train)

0:	learn: 0.6300112	total: 14.2ms	remaining: 7.09s
1:	learn: 0.5741290	total: 26.4ms	remaining: 6.57s
2:	learn: 0.5230950	total: 38.3ms	remaining: 6.34s
3:	learn: 0.4800382	total: 50.3ms	remaining: 6.24s
4:	learn: 0.4403254	total: 62.3ms	remaining: 6.17s
5:	learn: 0.4049130	total: 74.4ms	remaining: 6.12s
6:	learn: 0.3740459	total: 86.5ms	remaining: 6.09s
7:	learn: 0.3459914	total: 98.3ms	remaining: 6.05s
8:	learn: 0.3212827	total: 111ms	remaining: 6.04s
9:	learn: 0.2979507	total: 121ms	remaining: 5.93s
10:	learn: 0.2788469	total: 131ms	remaining: 5.82s
11:	learn: 0.2622651	total: 141ms	remaining: 5.74s
12:	learn: 0.2476561	total: 151ms	remaining: 5.65s
13:	learn: 0.2341966	total: 161ms	remaining: 5.59s
14:	learn: 0.2213328	total: 171ms	remaining: 5.53s
15:	learn: 0.2089498	total: 181ms	remaining: 5.47s
16:	learn: 0.1993485	total: 191ms	remaining: 5.42s
17:	learn: 0.1911051	total: 201ms	remaining: 5.38s
18:	learn: 0.1830265	total: 211ms	remaining: 5.35s
19:	learn: 0.1755564	total: 221ms

172:	learn: 0.0934468	total: 1.67s	remaining: 3.15s
173:	learn: 0.0934345	total: 1.67s	remaining: 3.14s
174:	learn: 0.0934145	total: 1.68s	remaining: 3.13s
175:	learn: 0.0933990	total: 1.69s	remaining: 3.12s
176:	learn: 0.0933845	total: 1.7s	remaining: 3.11s
177:	learn: 0.0933601	total: 1.71s	remaining: 3.1s
178:	learn: 0.0933383	total: 1.72s	remaining: 3.09s
179:	learn: 0.0933203	total: 1.73s	remaining: 3.08s
180:	learn: 0.0933110	total: 1.74s	remaining: 3.07s
181:	learn: 0.0932970	total: 1.75s	remaining: 3.06s
182:	learn: 0.0932868	total: 1.76s	remaining: 3.05s
183:	learn: 0.0932782	total: 1.77s	remaining: 3.04s
184:	learn: 0.0932692	total: 1.78s	remaining: 3.02s
185:	learn: 0.0932546	total: 1.79s	remaining: 3.02s
186:	learn: 0.0932453	total: 1.8s	remaining: 3.01s
187:	learn: 0.0932303	total: 1.8s	remaining: 3s
188:	learn: 0.0932159	total: 1.81s	remaining: 2.99s
189:	learn: 0.0932028	total: 1.82s	remaining: 2.98s
190:	learn: 0.0931912	total: 1.83s	remaining: 2.97s
191:	learn: 0.09317

348:	learn: 0.0913087	total: 3.31s	remaining: 1.43s
349:	learn: 0.0912978	total: 3.32s	remaining: 1.42s
350:	learn: 0.0912920	total: 3.33s	remaining: 1.41s
351:	learn: 0.0912756	total: 3.34s	remaining: 1.4s
352:	learn: 0.0912644	total: 3.35s	remaining: 1.39s
353:	learn: 0.0912562	total: 3.36s	remaining: 1.38s
354:	learn: 0.0912447	total: 3.37s	remaining: 1.38s
355:	learn: 0.0912348	total: 3.38s	remaining: 1.37s
356:	learn: 0.0912289	total: 3.39s	remaining: 1.36s
357:	learn: 0.0912219	total: 3.4s	remaining: 1.35s
358:	learn: 0.0912160	total: 3.4s	remaining: 1.34s
359:	learn: 0.0912070	total: 3.41s	remaining: 1.33s
360:	learn: 0.0911920	total: 3.42s	remaining: 1.32s
361:	learn: 0.0911824	total: 3.43s	remaining: 1.31s
362:	learn: 0.0911693	total: 3.44s	remaining: 1.3s
363:	learn: 0.0911595	total: 3.45s	remaining: 1.29s
364:	learn: 0.0911541	total: 3.46s	remaining: 1.28s
365:	learn: 0.0911481	total: 3.47s	remaining: 1.27s
366:	learn: 0.0911380	total: 3.48s	remaining: 1.26s
367:	learn: 0.09

0:	learn: 0.5965772	total: 13.1ms	remaining: 6.56s
1:	learn: 0.5142973	total: 24.5ms	remaining: 6.11s
2:	learn: 0.4387375	total: 35.3ms	remaining: 5.84s
3:	learn: 0.3714045	total: 45.5ms	remaining: 5.64s
4:	learn: 0.3138431	total: 55.3ms	remaining: 5.47s
5:	learn: 0.2661446	total: 65.4ms	remaining: 5.38s
6:	learn: 0.2261139	total: 75.8ms	remaining: 5.34s
7:	learn: 0.1931082	total: 85.9ms	remaining: 5.28s
8:	learn: 0.1658817	total: 96.2ms	remaining: 5.25s
9:	learn: 0.1431357	total: 107ms	remaining: 5.23s
10:	learn: 0.1247113	total: 117ms	remaining: 5.19s
11:	learn: 0.1092862	total: 127ms	remaining: 5.17s
12:	learn: 0.0967858	total: 137ms	remaining: 5.14s
13:	learn: 0.0859732	total: 147ms	remaining: 5.11s
14:	learn: 0.0768869	total: 157ms	remaining: 5.08s
15:	learn: 0.0690225	total: 167ms	remaining: 5.05s
16:	learn: 0.0623332	total: 177ms	remaining: 5.03s
17:	learn: 0.0568060	total: 187ms	remaining: 5s
18:	learn: 0.0520948	total: 197ms	remaining: 4.99s
19:	learn: 0.0479619	total: 207ms	r

174:	learn: 0.0126423	total: 1.66s	remaining: 3.09s
175:	learn: 0.0126366	total: 1.67s	remaining: 3.08s
176:	learn: 0.0126281	total: 1.68s	remaining: 3.07s
177:	learn: 0.0126216	total: 1.69s	remaining: 3.06s
178:	learn: 0.0125998	total: 1.7s	remaining: 3.05s
179:	learn: 0.0125890	total: 1.71s	remaining: 3.04s
180:	learn: 0.0125807	total: 1.72s	remaining: 3.03s
181:	learn: 0.0125723	total: 1.73s	remaining: 3.02s
182:	learn: 0.0125649	total: 1.74s	remaining: 3.01s
183:	learn: 0.0125561	total: 1.74s	remaining: 3s
184:	learn: 0.0125445	total: 1.75s	remaining: 2.99s
185:	learn: 0.0125369	total: 1.76s	remaining: 2.98s
186:	learn: 0.0125233	total: 1.77s	remaining: 2.97s
187:	learn: 0.0125077	total: 1.78s	remaining: 2.96s
188:	learn: 0.0125039	total: 1.79s	remaining: 2.95s
189:	learn: 0.0124927	total: 1.8s	remaining: 2.94s
190:	learn: 0.0124829	total: 1.81s	remaining: 2.93s
191:	learn: 0.0124743	total: 1.82s	remaining: 2.92s
192:	learn: 0.0124617	total: 1.83s	remaining: 2.91s
193:	learn: 0.012

...

351:	learn: 0.0592165	total: 3.27s	remaining: 1.37s
352:	learn: 0.0592114	total: 3.28s	remaining: 1.36s
353:	learn: 0.0592041	total: 3.29s	remaining: 1.35s
354:	learn: 0.0591966	total: 3.3s	remaining: 1.35s
355:	learn: 0.0591925	total: 3.31s	remaining: 1.34s
356:	learn: 0.0591861	total: 3.31s	remaining: 1.33s
357:	learn: 0.0591820	total: 3.32s	remaining: 1.32s
358:	learn: 0.0591759	total: 3.33s	remaining: 1.31s
359:	learn: 0.0591648	total: 3.34s	remaining: 1.3s
360:	learn: 0.0591556	total: 3.35s	remaining: 1.29s
361:	learn: 0.0591503	total: 3.36s	remaining: 1.28s
362:	learn: 0.0591408	total: 3.37s	remaining: 1.27s
363:	learn: 0.0591339	total: 3.38s	remaining: 1.26s
364:	learn: 0.0591270	total: 3.39s	remaining: 1.25s
365:	learn: 0.0591207	total: 3.4s	remaining: 1.24s
366:	learn: 0.0591123	total: 3.41s	remaining: 1.24s
367:	learn: 0.0591006	total: 3.42s	remaining: 1.23s
368:	learn: 0.0590962	total: 3.43s	remaining: 1.22s
369:	learn: 0.0590894	total: 3.43s	remaining: 1.21s
370:	learn: 0.0

0:	learn: 0.6257764	total: 15ms	remaining: 7.48s
1:	learn: 0.5651476	total: 28.9ms	remaining: 7.2s
2:	learn: 0.5085256	total: 43ms	remaining: 7.12s
3:	learn: 0.4633903	total: 56.4ms	remaining: 7s
4:	learn: 0.4234627	total: 70.7ms	remaining: 7s
5:	learn: 0.3840513	total: 84.6ms	remaining: 6.96s
6:	learn: 0.3526822	total: 94.5ms	remaining: 6.65s
7:	learn: 0.3229816	total: 104ms	remaining: 6.41s
8:	learn: 0.2984860	total: 114ms	remaining: 6.22s
9:	learn: 0.2753699	total: 125ms	remaining: 6.1s
10:	learn: 0.2558969	total: 135ms	remaining: 5.99s
11:	learn: 0.2379854	total: 144ms	remaining: 5.88s
12:	learn: 0.2226474	total: 154ms	remaining: 5.78s
13:	learn: 0.2101508	total: 164ms	remaining: 5.7s
14:	learn: 0.1979292	total: 174ms	remaining: 5.63s
15:	learn: 0.1869549	total: 184ms	remaining: 5.56s
16:	learn: 0.1774927	total: 193ms	remaining: 5.49s
17:	learn: 0.1694308	total: 203ms	remaining: 5.44s
18:	learn: 0.1621997	total: 214ms	remaining: 5.41s
19:	learn: 0.1545322	total: 224ms	remaining: 5.

173:	learn: 0.0756912	total: 1.65s	remaining: 3.09s
174:	learn: 0.0756692	total: 1.66s	remaining: 3.08s
175:	learn: 0.0756538	total: 1.67s	remaining: 3.07s
176:	learn: 0.0756379	total: 1.68s	remaining: 3.06s
177:	learn: 0.0756186	total: 1.69s	remaining: 3.05s
178:	learn: 0.0756015	total: 1.7s	remaining: 3.04s
179:	learn: 0.0755803	total: 1.71s	remaining: 3.04s
180:	learn: 0.0755621	total: 1.72s	remaining: 3.02s
181:	learn: 0.0755436	total: 1.73s	remaining: 3.01s
182:	learn: 0.0755217	total: 1.74s	remaining: 3s
183:	learn: 0.0755030	total: 1.74s	remaining: 3s
184:	learn: 0.0754736	total: 1.75s	remaining: 2.99s
185:	learn: 0.0754556	total: 1.76s	remaining: 2.98s
186:	learn: 0.0754395	total: 1.77s	remaining: 2.97s
187:	learn: 0.0754149	total: 1.78s	remaining: 2.96s
188:	learn: 0.0754039	total: 1.79s	remaining: 2.95s
189:	learn: 0.0753808	total: 1.8s	remaining: 2.94s
190:	learn: 0.0753634	total: 1.81s	remaining: 2.93s
191:	learn: 0.0753438	total: 1.82s	remaining: 2.92s
192:	learn: 0.075328

348:	learn: 0.0727054	total: 3.28s	remaining: 1.42s
349:	learn: 0.0726937	total: 3.29s	remaining: 1.41s
350:	learn: 0.0726851	total: 3.29s	remaining: 1.4s
351:	learn: 0.0726723	total: 3.3s	remaining: 1.39s
352:	learn: 0.0726589	total: 3.31s	remaining: 1.38s
353:	learn: 0.0726447	total: 3.32s	remaining: 1.37s
354:	learn: 0.0726371	total: 3.33s	remaining: 1.36s
355:	learn: 0.0726234	total: 3.34s	remaining: 1.35s
356:	learn: 0.0726058	total: 3.35s	remaining: 1.34s
357:	learn: 0.0725990	total: 3.36s	remaining: 1.33s
358:	learn: 0.0725809	total: 3.37s	remaining: 1.32s
359:	learn: 0.0725742	total: 3.38s	remaining: 1.31s
360:	learn: 0.0725603	total: 3.38s	remaining: 1.3s
361:	learn: 0.0725405	total: 3.39s	remaining: 1.29s
362:	learn: 0.0725202	total: 3.4s	remaining: 1.28s
363:	learn: 0.0725112	total: 3.41s	remaining: 1.27s
364:	learn: 0.0724904	total: 3.42s	remaining: 1.26s
365:	learn: 0.0724800	total: 3.43s	remaining: 1.26s
366:	learn: 0.0724662	total: 3.44s	remaining: 1.25s
367:	learn: 0.07

MultiOutputRegressor(estimator=<catboost.core.CatBoostClassifier object at 0x7f8db10a8c18>,
                     n_jobs=None)

# Run predictions on trained XGB model

In [28]:
#predictions from model and evaluation
x_test = val_df[model_vars]
y_pred = xgb_clf.predict(x_test)
y_true = val_df[base_classes]

l = 0
for i,c in enumerate(y_true.columns):
    loss = log_loss( y_true[c], y_pred[:,i] )
    l += loss if i != 0 else loss*2
    print(i,c,loss)
print(l/7.)

0 any 0.09340231172025013
1 epidural 0.012141253590899853
2 intraparenchymal 0.039596951835288635
3 intraventricular 0.02329420861999448
4 subarachnoid 0.062225838595083294
5 subdural 0.07753075725012087
0.05737051904741248


In [29]:
#predictions from model and evaluation
x_test = val_df[model_vars]
_y_pred = [c.predict_proba(x_test)[:,1:] for c in cb_clf.estimators_]
y_pred = np.concatenate(_y_pred,axis=-1)
y_true = val_df[base_classes]

l = 0
for i,c in enumerate(y_true.columns):
    loss = log_loss( y_true[c], y_pred[:,i] )
    l += loss if i != 0 else loss*2
    print(i,c,loss)
print(l/7.)

0 any 0.0936741594705755
1 epidural 0.012621494943967658
2 intraparenchymal 0.039732457939622046
3 intraventricular 0.023184423126184128
4 subarachnoid 0.06195888386520253
5 subdural 0.0781302903061038
0.057567981303175884


# Prepare submission with trained model

In [30]:
sub_path = 'data/submission_L2.csv'

data = {}
for test_df in test_dfs:
    x_test = test_df[model_vars]
    xgb_probas = xgb_clf.predict(x_test)
    cb_probas  = np.concatenate([c.predict_proba(x_test)[:,1:] for c in cb_clf.estimators_],axis=-1)
    
    probas = (xgb_probas + cb_probas) / 2.
    
    for idx, row in test_df.iterrows():
        file_id = row.fn
        for klass_index, klass in enumerate(base_classes):
            key =  file_id + '_' + klass
            if key not in data:
                data[key] = probas[idx, klass_index]
            else:
                data[key] += probas[idx, klass_index]

print('finish')                
                
final_prediction = []
for key in data:
    final_prediction.append([key, data[key] / 5])

df = pd.DataFrame(final_prediction, columns=['ID','Label'])
df.to_csv(sub_path, index=False)


finish


In [31]:
msg = '"L2 Stacking of ' + " ".join(arch_names) + ' ensembled with xgboost + catboost"'
msg

'"L2 Stacking of densenet121 densenet121-53_train_cpu_window_uint8 resnet101 resnet101-53_train_cpu_window_uint8 resnet18 resnet18-53_train_cpu_window_uint8 resnet34 resnet34-53_train_cpu_window_uint8 resnet50 resnext50_32x4d-53_train_cpu_window_uint8 ensembled with xgboost + catboost"'

In [ ]:
!kaggle competitions submit -c rsna-intracranial-hemorrhage-detection -f {sub_path} -m {msg}